** Let's see if having Unopposed as zero ends up being predictive at all...** FINGERS CROSSED!

In [1]:
import pandas as pd
import csv
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
%matplotlib inline
# from paramsearch import paramsearch
from itertools import product,chain

### Data Aggregation

#### Prepping Files 

Reading the CSV files

In [2]:
hm2016 = pd.read_csv('federalelections2016 (2).csv')
hm2014 = pd.read_csv('federalelections2014.csv')
hm2012 = pd.read_csv('Senate_Primary_2012.csv')
hm2010 = pd.read_csv('Primary2010.csv')
hm2008 = pd.read_csv('Primary2008.csv')
hm2006 = pd.read_csv('Primary2006.csv')
hm2004 = pd.read_csv('FederalElection2004.csv')


### Add back incumbent column!!

**Adding the year to each CSV for easier analysis**

In [3]:
hm2016['YEAR'] = '2016'
hm2014['YEAR'] = '2014'
hm2012['YEAR'] = '2012'
hm2010['YEAR'] = '2010'
hm2008['YEAR'] = '2008'
hm2006['YEAR'] = '2006'
hm2004['YEAR'] = '2004'

**Creating IDs I will later merge on**

In [4]:
hm2016["ID"] = hm2016["STATE"].map(str) + hm2016["YEAR"] +hm2016["PARTY"]
hm2016["ID2"] = hm2016["STATE"].map(str) + hm2016["YEAR"]

hm2014["ID"] = hm2014["STATE"].map(str) + hm2014["YEAR"] +hm2014["PARTY"]
hm2014["ID2"] = hm2014["STATE"].map(str) + hm2014["YEAR"]

hm2012["ID"] = hm2012["STATE"].map(str) + hm2012["YEAR"] +hm2012["PARTY"]
hm2012["ID2"] = hm2012["STATE"].map(str) + hm2012["YEAR"]

hm2010["ID"] = hm2010["STATE"].map(str) + hm2010["YEAR"] +hm2010["PARTY"]
hm2010["ID2"] = hm2010["STATE"].map(str) + hm2010["YEAR"]

hm2008["ID"] = hm2008["STATE"].map(str) + hm2008["YEAR"] +hm2008["PARTY"]
hm2008["ID2"] = hm2008["STATE"].map(str) + hm2008["YEAR"]

hm2006["ID"] = hm2006["STATE"].map(str) + hm2006["YEAR"] +hm2006["PARTY"]
hm2006["ID2"] = hm2006["STATE"].map(str) + hm2006["YEAR"]

hm2004["ID"] = hm2004["STATE"].map(str) + hm2004["YEAR"] +hm2004["PARTY"]
hm2004["ID2"] = hm2004["STATE"].map(str) + hm2004["YEAR"]


**Dropping weird columns & fixing weird merging errors**

In [5]:
hm2014 = hm2014.drop('Unnamed: 8', axis=1).drop('Unnamed: 9',axis=1)

In [6]:
# hm2012 = hm2012.drop('Unnamed: 9', axis=1)

In [7]:
# hm2010.columns = [col.strip() for col in hm2010.columns]

In [8]:
hm2012.columns = [col.strip() for col in hm2012.columns]

In [9]:
hm2014.columns = [col.strip() for col in hm2014.columns]

In [10]:
hm2016.columns = [col.strip() for col in hm2016.columns]

**Append files together**

In [11]:
hm = hm2016.append(hm2012).append(hm2004).append(hm2006).append(hm2008).append(hm2010).append(hm2014)

In [ ]:
# ## Want to make "Incument" versus not a binary count.  
# hm = hm.replace(to_replace = 'FULL TERM',value='(I)').replace(to_replace = 'UNEXPIRED TERM ',value='(I)').replace(to_replace=' (I)',value='(I)').replace(to_replace='UNEXPIRED TERM',value='(I)')

In [54]:
len(hm[hm['PRIMARY VOTES']== 'Unopposed'])

71

#### Party data

**Removing all third party candidates**

In [30]:
hm = hm.replace(to_replace = 'R*',value='R').replace(to_replace = 'R ',value='R').replace(to_replace = 'D*',value='D').replace(to_replace = 'D* ',value='D').replace(to_replace = 'D ',value='D')

In [31]:
hm_party = hm[hm['PARTY'].isin(['R','D'])]

**Throwing out rows that are interupting analysis**

In [56]:
hm_party2 = hm_party[~hm_party['PRIMARY VOTES'].isin(['*','#','14*','20*','1,040*','1,616**','Loser','Winner','Withdrew'])]

In [57]:
hm_party2 = hm_party2.replace(to_replace ='Unopposed', value=0)

In [58]:
hm_party2 = hm_party2[~hm_party2['TOTAL VOTES'].isin(['Party Votes:','Total State Votes:'])]

In [59]:
hm_party2 = hm_party2[~hm_party2['ID'].isin(['South Carolina2014R','Wyoming2008R'])]

**Find total votes for each party, each state **

In [48]:
#Didn't work :( It's missing a few states.
# hm_partytotal = hm_party[hm_party['TOTAL VOTES'].isin(['Party Votes:'])]
# hm_partytotal = hm_partytotal[['ID','PRIMARY VOTES']]

In [60]:
hm_partytotal = hm_party2[['ID','PRIMARY VOTES']]

In [62]:
hm_partytotal = hm_partytotal.dropna()

In [63]:
hm_partytotal["PRIMARY VOTES"]=hm_partytotal["PRIMARY VOTES"].astype(float)

In [64]:
hm_partytotal = hm_partytotal.groupby(['ID']).sum().reset_index()

In [65]:
hm_partytotal = hm_partytotal.rename(columns = {'PRIMARY VOTES':'Total Party Votes'})

**Finding How many people ran in the primary**

In [66]:
NumRunning = hm_party2.groupby(['ID']).count().reset_index()

In [67]:
NumRunning = NumRunning[['ID','PRIMARY VOTES','(I)']]

In [68]:
NumRunning['(I)'].value_counts()

0    276
1    168
Name: (I), dtype: int64

In [69]:
NumRunning = NumRunning.rename(columns = {'PRIMARY VOTES':'Candidate Count'})

**Finding max vote for each party, each state**

In [70]:
PartyWin = hm_party2[['ID','PRIMARY VOTES','STATE','YEAR','PARTY']]

In [71]:
PartyWin = PartyWin.dropna()

In [72]:
PartyWin["PRIMARY VOTES"]= PartyWin["PRIMARY VOTES"].astype(float)

In [73]:
PartyWin = PartyWin.groupby(['ID']).max().reset_index()

In [74]:
PartyWin = PartyWin.rename(columns = {'PRIMARY VOTES':'Primary Winner'})

-Group by state and party
-Tuple of the candidates aggregated 
-Winner, candidates, number of columns


**Merging Columns Back Together**

In [75]:
votes_merged = hm_partytotal.merge(NumRunning, left_on='ID', right_on='ID', how='left')

In [76]:
votes_merged = votes_merged.merge(PartyWin, left_on='ID', right_on='ID', how = 'left')

**Calcuating the total number of votes by losers**

In [77]:
votes_merged['Primary Loser Vote'] = votes_merged['Total Party Votes'] - votes_merged['Primary Winner']

#### Final Results Data

Creating column with final election results numbers

In [78]:
hm_finals = hm[hm['PARTY'].isin(['R','D'])]

In [79]:
hm_finals = hm_finals[['GENERAL VOTES','ID','ID2']]

In [80]:
hm_finals = hm_finals.dropna()

#### 3rd Party Data #Not Including this anymore

Creating column with how many people showed up to vote for the third party candidate in the primaries. 

In [81]:
hm_third = hm[~hm['PARTY'].isin(['R','D'])]

In [82]:
hm_third = hm_third[hm_third['TOTAL VOTES'].isin(['Party Votes:'])]

In [83]:
hm_third = hm_third[['PRIMARY VOTES','ID2']]

In [84]:
hm_third = hm_third.groupby(['ID2'])
hm_third = hm_third['PRIMARY VOTES'].agg([np.sum]).reset_index()

In [85]:
hm_third = hm_third.rename(columns = {'sum':'Third Party Votes'})

#### Merging

Merging the final results data and the third party data to the original dataframe.

In [86]:
hm_merged = hm_finals.merge(votes_merged, left_on='ID', right_on='ID', how='left')

In [ ]:
# hm_merged = hm_merged.merge(hm_third, left_on='ID2', right_on='ID2', how='left')

In [ ]:
# hm_merged['Third Party Votes'] = hm_merged['Third Party Votes'].fillna(value=0)

** Throwing out NaNs for races that didn't have primary data**

In [87]:
hm_merged = hm_merged.dropna()

**Making a loop for whether or not a candidate's party is in the oval office.**

In [88]:
Republican = ['2008','2006','2004','2002','1992','1990']

OfficeParty = []


for year in hm_merged['YEAR']:
    if year in Republican:
        OfficeParty.append('R')
        
    else:
        OfficeParty.append('D')
    
hm_merged['OFFICE PARTY'] = OfficeParty

** Create Column for whether or not the party is in office**

In [89]:
hm_merged['Party In House'] = np.where(hm_merged['PARTY']== hm_merged['OFFICE PARTY'], 1,0)

In [90]:
hm_model_ready = hm_merged.drop('ID',axis=1).drop('ID2',axis=1).drop('OFFICE PARTY',axis=1)

In [91]:
# hm_model_ready = hm_merged.drop('ID',axis=1).drop('ID2',axis=1).drop('Total Party Votes',axis=1).drop('YEAR',axis=1).drop('OFFICE PARTY',axis=1)

In [92]:
hm_model_ready = hm_model_ready[~hm_model_ready['GENERAL VOTES'].isin(['#'])]

In [93]:
hm_model_ready

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,STATE,YEAR,PARTY,Primary Loser Vote,Party In House
0,1.3351e+06,778851.0,5.0,1.0,505586.0,Alabama,2016,R,273265.0,0
1,748709,274423.0,2.0,0.0,153897.0,Alabama,2016,D,120526.0,1
2,138149,55293.0,4.0,1.0,39545.0,Alaska,2016,R,15748.0,0
3,36200,25318.0,2.0,0.0,15228.0,Alaska,2016,D,10090.0,1
4,1.35927e+06,591155.0,4.0,1.0,302532.0,Arizona,2016,R,288623.0,0
5,1.03124e+06,333586.0,1.0,0.0,333586.0,Arizona,2016,D,0.0,1
6,661984,389834.0,2.0,1.0,298039.0,Arkansas,2016,R,91795.0,0
7,400602,0.0,1.0,0.0,0.0,Arkansas,2016,D,0.0,1
8,7.54276e+06,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1
9,4.70142e+06,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1


In [94]:
hm_model_ready["GENERAL VOTES"]= hm_model_ready["GENERAL VOTES"].astype(float)
# hm_model_ready["Third Party Votes"]= hm_model_ready["Third Party Votes"].astype(float)
hm_model_ready["Party In House"]= hm_model_ready["Party In House"].astype(float)

### Modeling (All 50 States)

#### EDA & Futhur Cleaning

In [95]:
hm_model_ready.describe()

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,Primary Loser Vote,Party In House
count,3.690000e+02,3.690000e+02,369.000000,369.000000,3.690000e+02,3.690000e+02,369.000000
mean,1.120396e+06,3.879946e+05,3.219512,0.387534,2.702476e+05,1.177470e+05,0.495935
std,1.203574e+06,5.490480e+05,2.552290,0.487849,3.872883e+05,2.164664e+05,0.500662
min,2.937700e+04,0.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,2.846400e+05,7.813200e+04,1.000000,0.000000,5.587800e+04,0.000000e+00,0.000000
50%,7.919500e+05,2.189690e+05,3.000000,0.000000,1.459740e+05,3.312100e+04,0.000000
75%,1.479471e+06,5.216570e+05,4.000000,1.000000,3.466700e+05,1.381150e+05,1.000000
max,7.864624e+06,4.811300e+06,18.000000,1.000000,3.000689e+06,1.810611e+06,1.000000


In [96]:
hm_model_ready.sort_values(by=['GENERAL VOTES'])

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,STATE,YEAR,PARTY,Primary Loser Vote,Party In House
414,29377.0,14870.0,4.0,0.0,7200.0,Wyoming,2014,D,7670.0,1.0
3,36200.0,25318.0,2.0,0.0,15228.0,Alaska,2016,D,10090.0,1.0
141,53019.0,16850.0,3.0,0.0,9173.0,Wyoming,2012,D,7677.0,1.0
273,57671.0,24924.0,1.0,0.0,24924.0,Wyoming,2006,D,0.0,0.0
277,60045.0,30287.0,3.0,0.0,18035.0,Alaska,2010,D,12252.0,1.0
250,64417.0,40647.0,1.0,0.0,40647.0,North Dakota,2006,R,0.0,1.0
219,69734.0,11881.0,2.0,0.0,6110.0,Delaware,2006,R,5771.0,1.0
335,72699.0,22750.0,1.0,0.0,22750.0,Vermont,2010,R,0.0,0.0
131,72898.0,8442.0,2.0,0.0,6358.0,Vermont,2012,R,2084.0,0.0
206,75398.0,13364.0,3.0,0.0,9591.0,Vermont,2004,R,3773.0,1.0


In [97]:
#Looks like there are some rogue additional states...
len(hm_model_ready['STATE'].value_counts())

51

In [98]:
hm_model_ready['STATE'].unique()

##Virgin Islands not in state list
##Missouri counted twice due to space error

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kentucky', 'Maryland', 'Missouri', 'Nevada',
       'New Hampshire', 'New York', 'North Carolina', 'North Dakota',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina',
       'South Dakota', 'Utah', 'Vermont', 'Washington', 'Wisconsin',
       'Connecticut', 'Delaware', 'Maine', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri ', 'Montana', 'Nebraska',
       'New Jersey', 'New Mexico', 'Rhode Island', 'Tennessee', 'Texas',
       'Virginia', 'West Virginia', 'Wyoming', 'Kansas', 'Louisiana'],
      dtype=object)

In [152]:
#Dropping VI
hm_model_ready2 = hm_model_ready[~hm_model_ready['STATE'].isin(['Virgin Islands'])]

In [153]:
#Removing spaces

hm_model_ready2 = hm_model_ready2.replace(to_replace = 'Missouri ',value='Missouri')

In [154]:
len(hm_model_ready2['STATE'].unique())

50

In [155]:
hm_model_ready2['(I)'].value_counts()

0.0    226
1.0    143
Name: (I), dtype: int64

In [156]:
hm_model_ready2.groupby(['STATE']).sum().sort_values(['GENERAL VOTES'])

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,Primary Loser Vote,Party In House
STATE,,,,,,,
Wyoming,582045.0,311266.0,17.0,3.0,270989.0,40277.0,3.0
North Dakota,771729.0,350801.0,6.0,1.0,316588.0,34213.0,2.0
Alaska,880306.0,460223.0,23.0,3.0,318109.0,142114.0,4.0
Vermont,970052.0,278217.0,14.0,3.0,252712.0,25505.0,4.0
Kansas,1091200.0,438904.0,4.0,1.0,347891.0,91013.0,1.0
Rhode Island,1118726.0,339786.0,9.0,3.0,294531.0,45255.0,3.0
Delaware,1147959.0,143109.0,12.0,3.0,98441.0,44668.0,4.0
Louisiana,1191987.0,207481.0,6.0,1.0,163034.0,44447.0,1.0
South Dakota,1211988.0,74498.0,11.0,3.0,41377.0,33121.0,3.0


**Seeing if we throw out the rest of the unopposed if our score gets better...**
It doesn't!

In [157]:
# moreopp = hm_model_ready2[hm_model_ready2['Primary Loser Vote'] == 0]

In [158]:
# #Dropping Unnopposed to see if it makes it better...
# hm_model_ready2 = hm_model_ready2[~hm_model_ready['Primary Loser Vote'].isin([0])]

In [159]:
# hm_model_ready2.loc[hm_model_ready2['Primary Loser Vote' == 0.0, ['Primary Winner']] = 0]
# hm_model_ready2[hm_model_ready2['Primary Loser Vote']==0.0]
hm_model_ready2['Primary Winner'][hm_model_ready2['Primary Loser Vote'] == 0] = 0

/Users/vicky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [160]:
hm_model_ready2[hm_model_ready2 ['Primary Winner'] ==0]

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,STATE,YEAR,PARTY,Primary Loser Vote,Party In House
5,1031245.0,333586.0,1.0,0.0,0.0,Arizona,2016,D,0.0,1.0
7,400602.0,0.0,1.0,0.0,0.0,Arkansas,2016,D,0.0,1.0
10,1370710.0,262344.0,1.0,1.0,0.0,Colorado,2016,D,0.0,1.0
20,449017.0,119633.0,1.0,1.0,0.0,Idaho,2016,R,0.0,0.0
21,188249.0,26471.0,1.0,0.0,0.0,Idaho,2016,D,0.0,1.0
25,1158947.0,516183.0,1.0,0.0,0.0,Indiana,2016,D,0.0,1.0
26,926007.0,90089.0,1.0,1.0,0.0,Iowa,2016,R,0.0,0.0
52,354649.0,70374.0,1.0,0.0,0.0,New Hampshire,2016,D,0.0,1.0
54,4784218.0,0.0,1.0,1.0,0.0,New York,2016,D,0.0,1.0
55,1723920.0,0.0,1.0,0.0,0.0,New York,2016,R,0.0,0.0


In [161]:
hm_model_ready2

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,STATE,YEAR,PARTY,Primary Loser Vote,Party In House
0,1335104.0,778851.0,5.0,1.0,505586.0,Alabama,2016,R,273265.0,0.0
1,748709.0,274423.0,2.0,0.0,153897.0,Alabama,2016,D,120526.0,1.0
2,138149.0,55293.0,4.0,1.0,39545.0,Alaska,2016,R,15748.0,0.0
3,36200.0,25318.0,2.0,0.0,15228.0,Alaska,2016,D,10090.0,1.0
4,1359267.0,591155.0,4.0,1.0,302532.0,Arizona,2016,R,288623.0,0.0
5,1031245.0,333586.0,1.0,0.0,0.0,Arizona,2016,D,0.0,1.0
6,661984.0,389834.0,2.0,1.0,298039.0,Arkansas,2016,R,91795.0,0.0
7,400602.0,0.0,1.0,0.0,0.0,Arkansas,2016,D,0.0,1.0
8,7542759.0,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1.0
9,4701417.0,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1.0


#### Catboost (With All Inputs Included) ***THIS ONE PREVAILS!! :)

** Splitting x&y **

In [103]:
hm_model_ready2

,GENERAL VOTES,Total Party Votes,Candidate Count,(I),Primary Winner,STATE,YEAR,PARTY,Primary Loser Vote,Party In House
0,1335104.0,778851.0,5.0,1.0,505586.0,Alabama,2016,R,273265.0,0.0
1,748709.0,274423.0,2.0,0.0,153897.0,Alabama,2016,D,120526.0,1.0
2,138149.0,55293.0,4.0,1.0,39545.0,Alaska,2016,R,15748.0,0.0
3,36200.0,25318.0,2.0,0.0,15228.0,Alaska,2016,D,10090.0,1.0
4,1359267.0,591155.0,4.0,1.0,302532.0,Arizona,2016,R,288623.0,0.0
5,1031245.0,333586.0,1.0,0.0,333586.0,Arizona,2016,D,0.0,1.0
6,661984.0,389834.0,2.0,1.0,298039.0,Arkansas,2016,R,91795.0,0.0
7,400602.0,0.0,1.0,0.0,0.0,Arkansas,2016,D,0.0,1.0
8,7542759.0,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1.0
9,4701417.0,4811300.0,7.0,0.0,3000689.0,California,2016,D,1810611.0,1.0


In [162]:
ycat = hm_model_ready2['GENERAL VOTES'] 
Xcat = hm_model_ready2.drop('GENERAL VOTES',axis = 1)

** Splitting for T-T-S **

In [163]:
X_trainc, X_testc, y_trainc, y_testc = train_test_split(Xcat, ycat, test_size=0.3, random_state=42)

In [164]:
X_trainc.columns

Index(['Total Party Votes', 'Candidate Count', '(I)', 'Primary Winner',
       'STATE', 'YEAR', 'PARTY', 'Primary Loser Vote', 'Party In House'],
      dtype='object')

** Running Catboost **

In [165]:
from catboost import CatBoostRegressor, Pool

categorical_features_indices = np.where(X_trainc.dtypes != np.float)[0]
train_pool = Pool(X_trainc, y_trainc, cat_features=categorical_features_indices)
test_pool = Pool(X_testc, y_testc, cat_features=categorical_features_indices)


Catset=CatBoostRegressor(learning_rate = 0.01, depth = 10)
Cat = Catset.fit(X_trainc, y_trainc, cat_features=categorical_features_indices, plot=True, use_best_model=True)

You should provide test set for use best model. use_best_model parameter swiched to false value.


0:	learn: 1706152.4622453	total: 58.1ms	remaining: 58.1s
1:	learn: 1696840.2281737	total: 93.3ms	remaining: 46.6s
2:	learn: 1687231.6790903	total: 98.8ms	remaining: 32.8s
3:	learn: 1677198.4104707	total: 113ms	remaining: 28.1s
4:	learn: 1668084.9052241	total: 118ms	remaining: 23.6s
5:	learn: 1657284.9367231	total: 121ms	remaining: 20s
6:	learn: 1647916.1855094	total: 123ms	remaining: 17.4s
7:	learn: 1637702.3177310	total: 125ms	remaining: 15.5s
8:	learn: 1629153.2261290	total: 127ms	remaining: 14s
9:	learn: 1619290.9284340	total: 129ms	remaining: 12.8s
10:	learn: 1612566.8367314	total: 131ms	remaining: 11.8s
11:	learn: 1602372.3710189	total: 134ms	remaining: 11.1s
12:	learn: 1592807.2143642	total: 137ms	remaining: 10.4s
13:	learn: 1582970.9409605	total: 139ms	remaining: 9.8s
14:	learn: 1574252.1555916	total: 142ms	remaining: 9.29s
15:	learn: 1564712.5323285	total: 145ms	remaining: 8.94s
16:	learn: 1554737.9231651	total: 148ms	remaining: 8.55s
17:	learn: 1545603.7457375	total: 151ms	rem

158:	learn: 861040.6814778	total: 798ms	remaining: 4.22s
159:	learn: 858518.0235718	total: 807ms	remaining: 4.24s
160:	learn: 855729.3098607	total: 837ms	remaining: 4.36s
161:	learn: 854088.1743914	total: 843ms	remaining: 4.36s
162:	learn: 851881.9508774	total: 845ms	remaining: 4.34s
163:	learn: 850033.1677629	total: 848ms	remaining: 4.32s
164:	learn: 847956.3281416	total: 850ms	remaining: 4.3s
165:	learn: 845206.8963503	total: 857ms	remaining: 4.3s
166:	learn: 842845.1823039	total: 861ms	remaining: 4.29s
167:	learn: 840514.9424773	total: 866ms	remaining: 4.29s
168:	learn: 838826.4126294	total: 869ms	remaining: 4.27s
169:	learn: 836706.3309473	total: 881ms	remaining: 4.3s
170:	learn: 834800.6517400	total: 885ms	remaining: 4.29s
171:	learn: 832404.8071277	total: 891ms	remaining: 4.29s
172:	learn: 830156.1416630	total: 895ms	remaining: 4.28s
173:	learn: 828033.0019285	total: 904ms	remaining: 4.29s
174:	learn: 825742.5900259	total: 910ms	remaining: 4.29s
175:	learn: 824018.6249418	total: 

MetricVisualizer(data={'catboost_info': {'path': 'catboost_info', 'name': 'catboost_info', 'content': {'passed_iterations': 188, 'total_iterations': 1000, 'data': {'iterations': [{'learn': [1706152.462], 'iteration': 0, 'passed_time': 0.05812973756, 'remaining_time': 58.07160783}, {'learn': [1696840.228], 'iteration': 1, 'passed_time': 0.09329306689, 'remaining_time': 46.55324038}, {'learn': [1687231.679], 'iteration': 2, 'passed_time': 0.09882827015, 'remaining_time': 32.84392845}, {'learn': [1677198.41], 'iteration': 3, 'passed_time': 0.1126566905, 'remaining_time': 28.05151595}, {'learn': [1668084.905], 'iteration': 4, 'passed_time': 0.118346665, 'remaining_time': 23.55098633}, {'learn': [1657284.937], 'iteration': 5, 'passed_time': 0.120637388, 'remaining_time': 19.98559395}, {'learn': [1647916.186], 'iteration': 6, 'passed_time': 0.1228393366, 'remaining_time': 17.42563733}, {'learn': [1637702.318], 'iteration': 7, 'passed_time': 0.1252130254, 'remaining_time': 15.52641515}, {'learn': [1629153.226], 'iteration': 8, 'passed_time': 0.1272323731, 'remaining_time': 14.00969797}, {'learn': [1619290.928], 'iteration': 9, 'passed_time': 0.1291615263, 'remaining_time': 12.7869911}, {'learn': [1612566.837], 'iteration': 10, 'passed_time': 0.1311198841, 'remaining_time': 11.78886958}, {'learn': [1602372.371], 'iteration': 11, 'passed_time': 0.134496145, 'remaining_time': 11.07351594}, {'learn': [1592807.214], 'iteration': 12, 'passed_time': 0.1367663983, 'remaining_time': 10.38372578}, {'learn': [1582970.941], 'iteration': 13, 'passed_time': 0.1391478533, 'remaining_time': 9.799984523}, {'learn': [1574252.156], 'iteration': 14, 'passed_time': 0.1415130146, 'remaining_time': 9.292687956}, {'learn': [1564712.532], 'iteration': 15, 'passed_time': 0.1453744545, 'remaining_time': 8.940528949}, {'learn': [1554737.923], 'iteration': 16, 'passed_time': 0.1478310802, 'remaining_time': 8.548114816}, {'learn': [1545603.746], 'iteration': 17, 'passed_time': 0.1512630647, 'remaining_time': 8.252240531}, {'learn': [1536542.032], 'iteration': 18, 'passed_time': 0.1537135553, 'remaining_time': 7.936473566}, {'learn': [1527297.17], 'iteration': 19, 'passed_time': 0.1568974275, 'remaining_time': 7.687973948}, {'learn': [1518086.81], 'iteration': 20, 'passed_time': 0.1603168291, 'remaining_time': 7.473817892}, {'learn': [1508944.294], 'iteration': 21, 'passed_time': 0.1623269055, 'remaining_time': 7.2161688}, {'learn': [1500256.334], 'iteration': 22, 'passed_time': 0.1655304888, 'remaining_time': 7.031447284}, {'learn': [1491501.563], 'iteration': 23, 'passed_time': 0.1676638566, 'remaining_time': 6.818330168}, {'learn': [1483177.092], 'iteration': 24, 'passed_time': 0.1700461735, 'remaining_time': 6.631800766}, {'learn': [1475019.798], 'iteration': 25, 'passed_time': 0.1727697945, 'remaining_time': 6.472222303}, {'learn': [1467112.782], 'iteration': 26, 'passed_time': 0.1772858655, 'remaining_time': 6.388857302}, {'learn': [1458529.469], 'iteration': 27, 'passed_time': 0.1795217608, 'remaining_time': 6.231969695}, {'learn': [1450272.039], 'iteration': 28, 'passed_time': 0.185194741, 'remaining_time': 6.200830812}, {'learn': [1442217.127], 'iteration': 29, 'passed_time': 0.189562676, 'remaining_time': 6.129193191}, {'learn': [1434884.48], 'iteration': 30, 'passed_time': 0.2037173968, 'remaining_time': 6.367811532}, {'learn': [1427326.666], 'iteration': 31, 'passed_time': 0.2083768201, 'remaining_time': 6.303398807}, {'learn': [1418977.321], 'iteration': 32, 'passed_time': 0.2111688908, 'remaining_time': 6.187888407}, {'learn': [1410834.711], 'iteration': 33, 'passed_time': 0.2131215232, 'remaining_time': 6.05515857}, {'learn': [1404413.745], 'iteration': 34, 'passed_time': 0.2163679338, 'remaining_time': 5.965573033}, {'learn': [1396697.082], 'iteration': 35, 'passed_time': 0.2185861388, 'remaining_time': 5.853251049}, {'learn': [1389529.284], 'iteration': 36, 'passed_time': 0.2215645472, 'remaining_time': 5.766666459}, {'learn': [1382534.143], 'iter

189:	learn: 796471.9762471	total: 983ms	remaining: 4.19s
190:	learn: 794325.1129842	total: 998ms	remaining: 4.23s
191:	learn: 792203.7739979	total: 1s	remaining: 4.22s
192:	learn: 790888.5618009	total: 1s	remaining: 4.2s
193:	learn: 788740.1406827	total: 1.01s	remaining: 4.18s
194:	learn: 786547.1619139	total: 1.01s	remaining: 4.17s
195:	learn: 784501.2933198	total: 1.01s	remaining: 4.16s
196:	learn: 782298.0262366	total: 1.03s	remaining: 4.21s
197:	learn: 780936.5925094	total: 1.04s	remaining: 4.21s
198:	learn: 779370.2477955	total: 1.04s	remaining: 4.2s
199:	learn: 777508.5008431	total: 1.05s	remaining: 4.19s
200:	learn: 775500.8409715	total: 1.05s	remaining: 4.18s
201:	learn: 773548.1058727	total: 1.06s	remaining: 4.19s
202:	learn: 771728.7566165	total: 1.06s	remaining: 4.18s
203:	learn: 769824.5283224	total: 1.08s	remaining: 4.21s
204:	learn: 768597.9190236	total: 1.08s	remaining: 4.2s
205:	learn: 766943.4693288	total: 1.08s	remaining: 4.18s
206:	learn: 765103.8776078	total: 1.09s	

338:	learn: 620661.3175122	total: 1.98s	remaining: 3.86s
339:	learn: 620241.9853301	total: 1.98s	remaining: 3.85s
340:	learn: 619269.4189767	total: 2.01s	remaining: 3.88s
341:	learn: 618409.0988099	total: 2.01s	remaining: 3.86s
342:	learn: 617670.2978025	total: 2.01s	remaining: 3.85s
343:	learn: 616839.0227786	total: 2.02s	remaining: 3.85s
344:	learn: 616165.1518402	total: 2.02s	remaining: 3.84s
345:	learn: 615071.2175388	total: 2.03s	remaining: 3.83s
346:	learn: 614573.8349326	total: 2.03s	remaining: 3.82s
347:	learn: 613857.4920952	total: 2.03s	remaining: 3.81s
348:	learn: 613648.4107467	total: 2.03s	remaining: 3.79s
349:	learn: 612941.4127583	total: 2.04s	remaining: 3.78s
350:	learn: 612499.3257882	total: 2.04s	remaining: 3.77s
351:	learn: 612220.1917691	total: 2.04s	remaining: 3.76s
352:	learn: 611321.3657288	total: 2.04s	remaining: 3.75s
353:	learn: 610479.1652014	total: 2.05s	remaining: 3.73s
354:	learn: 609560.8745805	total: 2.05s	remaining: 3.72s
355:	learn: 609506.8382313	tota

493:	learn: 536030.9302161	total: 2.8s	remaining: 2.87s
494:	learn: 535835.3940288	total: 2.8s	remaining: 2.86s
495:	learn: 535358.9227069	total: 2.83s	remaining: 2.87s
496:	learn: 534659.8699942	total: 2.86s	remaining: 2.89s
497:	learn: 534448.5016708	total: 2.86s	remaining: 2.88s
498:	learn: 534372.3690418	total: 2.86s	remaining: 2.87s
499:	learn: 533529.5573441	total: 2.87s	remaining: 2.87s
500:	learn: 533320.3938033	total: 2.87s	remaining: 2.85s
501:	learn: 532028.1753865	total: 2.89s	remaining: 2.86s
502:	learn: 531739.6944322	total: 2.89s	remaining: 2.85s
503:	learn: 531674.8167069	total: 2.89s	remaining: 2.85s
504:	learn: 531461.6537144	total: 2.89s	remaining: 2.84s
505:	learn: 530458.3237556	total: 2.9s	remaining: 2.83s
506:	learn: 530247.0057312	total: 2.9s	remaining: 2.82s
507:	learn: 529642.1214810	total: 2.93s	remaining: 2.84s
508:	learn: 529070.7776652	total: 2.94s	remaining: 2.84s
509:	learn: 528973.4483518	total: 2.94s	remaining: 2.83s
510:	learn: 528645.3318703	total: 2

647:	learn: 466994.4842608	total: 4.04s	remaining: 2.19s
648:	learn: 466341.2970003	total: 4.06s	remaining: 2.19s
649:	learn: 466260.4272787	total: 4.06s	remaining: 2.19s
650:	learn: 466160.3359475	total: 4.07s	remaining: 2.18s
651:	learn: 466058.4318408	total: 4.07s	remaining: 2.17s
652:	learn: 465524.9409957	total: 4.09s	remaining: 2.17s
653:	learn: 465141.5964997	total: 4.1s	remaining: 2.17s
654:	learn: 464917.7437790	total: 4.1s	remaining: 2.16s
655:	learn: 464471.2795351	total: 4.11s	remaining: 2.16s
656:	learn: 464441.0474497	total: 4.12s	remaining: 2.15s
657:	learn: 463998.3554702	total: 4.12s	remaining: 2.14s
658:	learn: 463849.0244861	total: 4.12s	remaining: 2.13s
659:	learn: 463273.8470661	total: 4.13s	remaining: 2.13s
660:	learn: 462808.3270722	total: 4.14s	remaining: 2.12s
661:	learn: 462250.0716295	total: 4.17s	remaining: 2.13s
662:	learn: 462127.6597472	total: 4.18s	remaining: 2.12s
663:	learn: 461763.6333250	total: 4.18s	remaining: 2.12s
664:	learn: 461680.5665471	total:

801:	learn: 406576.3426510	total: 6.4s	remaining: 1.58s
802:	learn: 406113.3479759	total: 6.42s	remaining: 1.57s
803:	learn: 405683.3085094	total: 6.45s	remaining: 1.57s
804:	learn: 404951.7492565	total: 6.47s	remaining: 1.57s
805:	learn: 404512.3832168	total: 6.49s	remaining: 1.56s
806:	learn: 404148.2324217	total: 6.51s	remaining: 1.56s
807:	learn: 403938.3152945	total: 6.53s	remaining: 1.55s
808:	learn: 403205.6495801	total: 6.54s	remaining: 1.54s
809:	learn: 402827.1898665	total: 6.56s	remaining: 1.54s
810:	learn: 402201.7874180	total: 6.57s	remaining: 1.53s
811:	learn: 401706.4183442	total: 6.58s	remaining: 1.52s
812:	learn: 400855.5445863	total: 6.61s	remaining: 1.52s
813:	learn: 400483.3493857	total: 6.62s	remaining: 1.51s
814:	learn: 400242.9841851	total: 6.64s	remaining: 1.51s
815:	learn: 400034.7434742	total: 6.66s	remaining: 1.5s
816:	learn: 399680.6782375	total: 6.68s	remaining: 1.5s
817:	learn: 399336.3877407	total: 6.69s	remaining: 1.49s
818:	learn: 399060.1838086	total: 

948:	learn: 363121.0202127	total: 9.6s	remaining: 516ms
949:	learn: 362868.1523395	total: 9.63s	remaining: 507ms
950:	learn: 362627.4217146	total: 9.65s	remaining: 497ms
951:	learn: 362242.3934823	total: 9.68s	remaining: 488ms
952:	learn: 362110.1699065	total: 9.7s	remaining: 478ms
953:	learn: 361986.5066185	total: 9.72s	remaining: 469ms
954:	learn: 361510.5156731	total: 9.74s	remaining: 459ms
955:	learn: 361348.1754586	total: 9.76s	remaining: 449ms
956:	learn: 361207.3333218	total: 9.78s	remaining: 439ms
957:	learn: 361123.9843225	total: 9.79s	remaining: 429ms
958:	learn: 361038.7840330	total: 9.81s	remaining: 419ms
959:	learn: 360721.2453195	total: 9.83s	remaining: 410ms
960:	learn: 360155.0603524	total: 9.86s	remaining: 400ms
961:	learn: 360060.9768366	total: 9.88s	remaining: 390ms
962:	learn: 359379.2313739	total: 9.89s	remaining: 380ms
963:	learn: 359229.7505732	total: 9.91s	remaining: 370ms
964:	learn: 359046.7553638	total: 9.93s	remaining: 360ms
965:	learn: 358926.0071592	total:

In [166]:
## Checking out feature importance for this model. 

feature_importances = Cat.get_feature_importance(train_pool)
feature_names = X_trainc.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Total Party Votes: 49.5203542825143
STATE: 19.324826235312624
Primary Winner: 16.474285799606406
(I): 4.269804202048356
YEAR: 3.6938600208243857
Primary Loser Vote: 2.4538223779285344
PARTY: 2.4052426398301656
Candidate Count: 1.5503943294431999
Party In House: 0.3074101124920372


In [167]:
Cat.eval_metrics(data = test_pool, metrics =['RMSE'], plot = True)

{'RMSE': [1449168.8989154273,
  1438923.6160298362,
  1428643.935977522,
  1418263.2401577544,
  1410374.8026007654,
  1398505.8480239615,
  1388658.9345129752,
  1377697.0966170197,
  1369198.646085112,
  1357759.1956284523,
  1350570.097435111,
  1339946.8705055774,
  1328916.111928917,
  1319330.525027069,
  1311155.8556834492,
  1301391.2703681318,
  1290899.624592532,
  1281664.8527232357,
  1271343.7069139103,
  1262289.2988585995,
  1252423.0834729904,
  1242473.5122611234,
  1232682.3707755352,
  1222906.2337088059,
  1214270.2362101702,
  1205311.903883515,
  1196977.087838176,
  1188442.3165276775,
  1180146.4767367917,
  1171588.7963449182,
  1164119.5351010219,
  1156817.9306824359,
  1148728.0447405796,
  1140775.664213473,
  1133948.3842214812,
  1125162.287103678,
  1119002.4440588842,
  1111963.9860897227,
  1103849.1589682838,
  1097994.7714933269,
  1090544.5871957275,
  1083099.268975268,
  1075619.1461032466,
  1068162.876507864,
  1061225.0521691565,
  1056617.6777

MetricVisualizer(data={'catboost_info': {'path': 'catboost_info', 'name': 'catboost_info', 'content': {'passed_iterations': 999, 'total_iterations': 1000, 'data': {'iterations': [{'iteration': 0, 'eval_dataset': [1449168.899]}, {'iteration': 1, 'eval_dataset': [1438923.616]}, {'iteration': 2, 'eval_dataset': [1428643.936]}, {'iteration': 3, 'eval_dataset': [1418263.24]}, {'iteration': 4, 'eval_dataset': [1410374.803]}, {'iteration': 5, 'eval_dataset': [1398505.848]}, {'iteration': 6, 'eval_dataset': [1388658.935]}, {'iteration': 7, 'eval_dataset': [1377697.097]}, {'iteration': 8, 'eval_dataset': [1369198.646]}, {'iteration': 9, 'eval_dataset': [1357759.196]}, {'iteration': 10, 'eval_dataset': [1350570.097]}, {'iteration': 11, 'eval_dataset': [1339946.871]}, {'iteration': 12, 'eval_dataset': [1328916.112]}, {'iteration': 13, 'eval_dataset': [1319330.525]}, {'iteration': 14, 'eval_dataset': [1311155.856]}, {'iteration': 15, 'eval_dataset': [1301391.27]}, {'iteration': 16, 'eval_dataset': [1290899.625]}, {'iteration': 17, 'eval_dataset': [1281664.853]}, {'iteration': 18, 'eval_dataset': [1271343.707]}, {'iteration': 19, 'eval_dataset': [1262289.299]}, {'iteration': 20, 'eval_dataset': [1252423.083]}, {'iteration': 21, 'eval_dataset': [1242473.512]}, {'iteration': 22, 'eval_dataset': [1232682.371]}, {'iteration': 23, 'eval_dataset': [1222906.234]}, {'iteration': 24, 'eval_dataset': [1214270.236]}, {'iteration': 25, 'eval_dataset': [1205311.904]}, {'iteration': 26, 'eval_dataset': [1196977.088]}, {'iteration': 27, 'eval_dataset': [1188442.317]}, {'iteration': 28, 'eval_dataset': [1180146.477]}, {'iteration': 29, 'eval_dataset': [1171588.796]}, {'iteration': 30, 'eval_dataset': [1164119.535]}, {'iteration': 31, 'eval_dataset': [1156817.931]}, {'iteration': 32, 'eval_dataset': [1148728.045]}, {'iteration': 33, 'eval_dataset': [1140775.664]}, {'iteration': 34, 'eval_dataset': [1133948.384]}, {'iteration': 35, 'eval_dataset': [1125162.287]}, {'iteration': 36, 'eval_dataset': [1119002.444]}, {'iteration': 37, 'eval_dataset': [1111963.986]}, {'iteration': 38, 'eval_dataset': [1103849.159]}, {'iteration': 39, 'eval_dataset': [1097994.771]}, {'iteration': 40, 'eval_dataset': [1090544.587]}, {'iteration': 41, 'eval_dataset': [1083099.269]}, {'iteration': 42, 'eval_dataset': [1075619.146]}, {'iteration': 43, 'eval_dataset': [1068162.877]}, {'iteration': 44, 'eval_dataset': [1061225.052]}, {'iteration': 45, 'eval_dataset': [1056617.678]}, {'iteration': 46, 'eval_dataset': [1049079.253]}, {'iteration': 47, 'eval_dataset': [1041741.619]}, {'iteration': 48, 'eval_dataset': [1034751.155]}, {'iteration': 49, 'eval_dataset': [1028349.911]}, {'iteration': 50, 'eval_dataset': [1022102.668]}, {'iteration': 51, 'eval_dataset': [1015445.666]}, {'iteration': 52, 'eval_dataset': [1008552.514]}, {'iteration': 53, 'eval_dataset': [1003324.773]}, {'iteration': 54, 'eval_dataset': [996448.6089]}, {'iteration': 55, 'eval_dataset': [990445.2319]}, {'iteration': 56, 'eval_dataset': [984005.8505]}, {'iteration': 57, 'eval_dataset': [977936.7973]}, {'iteration': 58, 'eval_dataset': [972411.2873]}, {'iteration': 59, 'eval_dataset': [965651.7077]}, {'iteration': 60, 'eval_dataset': [960542.0633]}, {'iteration': 61, 'eval_dataset': [954640.4193]}, {'iteration': 62, 'eval_dataset': [950530.9298]}, {'iteration': 63, 'eval_dataset': [944739.4347]}, {'iteration': 64, 'eval_dataset': [939267.3976]}, {'iteration': 65, 'eval_dataset': [933548.0985]}, {'iteration': 66, 'eval_dataset': [927679.1981]}, {'iteration': 67, 'eval_dataset': [922480.5282]}, {'iteration': 68, 'eval_dataset': [916745.9712]}, {'iteration': 69, 'eval_dataset': [911307.7046]}, {'iteration': 70, 'eval_dataset': [906114.5634]}, {'iteration': 71, 'eval_dataset': [901300.796]}, {'iteration': 72, 'eval_dataset': [896304.3663]}, {'iteration': 73, 'eval_dataset': [891388.8687]}, {'iteration': 74, 'eval_dataset': [886063.0328]}, {'iteration': 75, 'eval_dataset': [880609.5033]}, {'iteration': 76, 'eval_dataset': 

In [168]:
Cat.eval_metrics(data = test_pool, metrics =['R2'], plot = True)

{'R2': [0.004788226750846447,
  0.009492245243488129,
  0.013514223865930819,
  0.017911366811277363,
  0.020518645733014207,
  0.025705570201292605,
  0.02959488013792777,
  0.03485921171703443,
  0.03715042457101414,
  0.042349688775589533,
  0.04286788714938017,
  0.049066709524073215,
  0.05482470853265453,
  0.05941933945288114,
  0.06249788123384703,
  0.0681190824902348,
  0.07445401269640683,
  0.07950990618481069,
  0.08485194875788082,
  0.0903193014142678,
  0.09571733583489694,
  0.10119602249672222,
  0.10688121140032936,
  0.11239376295985448,
  0.11731289357185104,
  0.1221162891767319,
  0.12741854724832402,
  0.13253575547276752,
  0.13805234593632165,
  0.14379133751930084,
  0.14931321770077544,
  0.15408043180171982,
  0.15954951901301617,
  0.16486747146952851,
  0.16875548486615122,
  0.1748444936398862,
  0.17837586971732255,
  0.1823098572883507,
  0.1875368261034409,
  0.19121897123197085,
  0.19616043599972022,
  0.2015436964270909,
  0.2064642292043941,
  0.2

MetricVisualizer(data={'catboost_info': {'path': 'catboost_info', 'name': 'catboost_info', 'content': {'passed_iterations': 999, 'total_iterations': 1000, 'data': {'iterations': [{'iteration': 0, 'eval_dataset': [0.004788226751]}, {'iteration': 1, 'eval_dataset': [0.009492245243]}, {'iteration': 2, 'eval_dataset': [0.01351422387]}, {'iteration': 3, 'eval_dataset': [0.01791136681]}, {'iteration': 4, 'eval_dataset': [0.02051864573]}, {'iteration': 5, 'eval_dataset': [0.0257055702]}, {'iteration': 6, 'eval_dataset': [0.02959488014]}, {'iteration': 7, 'eval_dataset': [0.03485921172]}, {'iteration': 8, 'eval_dataset': [0.03715042457]}, {'iteration': 9, 'eval_dataset': [0.04234968878]}, {'iteration': 10, 'eval_dataset': [0.04286788715]}, {'iteration': 11, 'eval_dataset': [0.04906670952]}, {'iteration': 12, 'eval_dataset': [0.05482470853]}, {'iteration': 13, 'eval_dataset': [0.05941933945]}, {'iteration': 14, 'eval_dataset': [0.06249788123]}, {'iteration': 15, 'eval_dataset': [0.06811908249]}, {'iteration': 16, 'eval_dataset': [0.0744540127]}, {'iteration': 17, 'eval_dataset': [0.07950990618]}, {'iteration': 18, 'eval_dataset': [0.08485194876]}, {'iteration': 19, 'eval_dataset': [0.09031930141]}, {'iteration': 20, 'eval_dataset': [0.09571733583]}, {'iteration': 21, 'eval_dataset': [0.1011960225]}, {'iteration': 22, 'eval_dataset': [0.1068812114]}, {'iteration': 23, 'eval_dataset': [0.112393763]}, {'iteration': 24, 'eval_dataset': [0.1173128936]}, {'iteration': 25, 'eval_dataset': [0.1221162892]}, {'iteration': 26, 'eval_dataset': [0.1274185472]}, {'iteration': 27, 'eval_dataset': [0.1325357555]}, {'iteration': 28, 'eval_dataset': [0.1380523459]}, {'iteration': 29, 'eval_dataset': [0.1437913375]}, {'iteration': 30, 'eval_dataset': [0.1493132177]}, {'iteration': 31, 'eval_dataset': [0.1540804318]}, {'iteration': 32, 'eval_dataset': [0.159549519]}, {'iteration': 33, 'eval_dataset': [0.1648674715]}, {'iteration': 34, 'eval_dataset': [0.1687554849]}, {'iteration': 35, 'eval_dataset': [0.1748444936]}, {'iteration': 36, 'eval_dataset': [0.1783758697]}, {'iteration': 37, 'eval_dataset': [0.1823098573]}, {'iteration': 38, 'eval_dataset': [0.1875368261]}, {'iteration': 39, 'eval_dataset': [0.1912189712]}, {'iteration': 40, 'eval_dataset': [0.196160436]}, {'iteration': 41, 'eval_dataset': [0.2015436964]}, {'iteration': 42, 'eval_dataset': [0.2064642292]}, {'iteration': 43, 'eval_dataset': [0.211429861]}, {'iteration': 44, 'eval_dataset': [0.2167552498]}, {'iteration': 45, 'eval_dataset': [0.2182362551]}, {'iteration': 46, 'eval_dataset': [0.2235988789]}, {'iteration': 47, 'eval_dataset': [0.2288208135]}, {'iteration': 48, 'eval_dataset': [0.2339504681]}, {'iteration': 49, 'eval_dataset': [0.238562448]}, {'iteration': 50, 'eval_dataset': [0.243463306]}, {'iteration': 51, 'eval_dataset': [0.248456322]}, {'iteration': 52, 'eval_dataset': [0.2536402224]}, {'iteration': 53, 'eval_dataset': [0.2578756558]}, {'iteration': 54, 'eval_dataset': [0.2638912377]}, {'iteration': 55, 'eval_dataset': [0.2685142343]}, {'iteration': 56, 'eval_dataset': [0.2737526126]}, {'iteration': 57, 'eval_dataset': [0.2787261295]}, {'iteration': 58, 'eval_dataset': [0.2827926253]}, {'iteration': 59, 'eval_dataset': [0.2891862691]}, {'iteration': 60, 'eval_dataset': [0.2933049684]}, {'iteration': 61, 'eval_dataset': [0.2984536089]}, {'iteration': 62, 'eval_dataset': [0.3014844471]}, {'iteration': 63, 'eval_dataset': [0.3060466859]}, {'iteration': 64, 'eval_dataset': [0.3103955677]}, {'iteration': 65, 'eval_dataset': [0.3156699611]}, {'iteration': 66, 'eval_dataset': [0.3209942066]}, {'iteration': 67, 'eval_dataset': [0.3251149797]}, {'iteration': 68, 'eval_dataset': [0.3299261281]}, {'iteration': 69, 'eval_dataset': [0.3344587241]}, {'iteration': 70, 'eval_dataset': [0.3386992829]}, {'iteration': 71, 'eval_dataset': [0.3427332225]}, {'iteration': 72, 'eval_dataset': [0.3469963931]}, {'iteration': 73, 'eval_dataset': [0.3519765984]}, {'iteration': 74, 'eval_dataset': [0.357370

In [169]:
Cat.score(X_testc,y_testc)

528703.0217896649

#### Catboost (Testing changing inputs)

**...Don't think it makes a difference because full list just crunches down unecessary inputs.**

In [ ]:
hm_model_ready2.columns

In [ ]:
ycat2 = hm_model_ready2['GENERAL VOTES'] 
Xcat2 = hm_model_ready2[['Primary Winner','Primary Loser Vote','STATE','PARTY']]

In [ ]:
X_trainc2, X_testc2, y_trainc2, y_testc2 = train_test_split(Xcat2, ycat2, test_size=0.2, random_state=42)

In [ ]:
categorical_features_indices = np.where(X_trainc2.dtypes != np.float)[0]
train_pool = Pool(X_trainc2, y_trainc2, cat_features=categorical_features_indices)
test_pool = Pool(X_testc2, y_testc2, cat_features=categorical_features_indices)


Cat = Catset.fit(X_trainc2, y_trainc2, cat_features=categorical_features_indices);

In [ ]:
Cat.score(X_testc2, y_testc2)

In [ ]:
Cat.eval_metrics(data = test_pool, metrics =['R2'], plot = True)

#### Linear Regression

In [ ]:
sns.pairplot(hm_model_ready, size = 1.2, aspect=1.5)

In [ ]:
ylr = hm_model_ready['GENERAL VOTES'] 
Xlr = hm_model_ready[['Primary Winner','Primary Loser Vote','Party In House','(I)']]

In [ ]:
Xlr

In [ ]:
X_trainlr, X_testlr, y_trainlr, y_testlr = train_test_split(Xlr, ylr, test_size=0.3, random_state=42)

In [ ]:
#Checking the predictability of the model with this alpha = 1
EN = ElasticNet(1, l1_ratio=.5)
LR = EN.fit(X_trainlr, y_trainlr)
rsq = LR.score(X_testlr, y_testlr)
adj_rsq = 1 - (1-rsq)*(len(ylr)-1)/(len(ylr)-Xlr.shape[1]-1)

preds = LR.predict(X_testlr)
rmse = np.sqrt(MSE(preds, y_testlr))

print(rsq)
print(adj_rsq)
print(rmse)


#### Linear Regression (Logged)

In [ ]:
hm_model_log = hm_model_ready

hm_model_log['Primary Winner']=np.log(hm_model_log['Primary Winner'])
hm_model_log['Primary Loser Vote']=np.log(hm_model_log['Primary Loser Vote'])
hm_model_log['Third Party Votes']=np.log(hm_model_log['Third Party Votes'])
hm_model_log['GENERAL VOTES']=np.log(hm_model_log['GENERAL VOTES'])

In [ ]:
ylr2 = hm_model_log['GENERAL VOTES'] 
Xlr2 = hm_model_log[['Primary Winner','Primary Loser Vote','Third Party Votes','Party In House','(I)']]

In [ ]:
ylr2

In [ ]:
Xlr2 = Xlr2.replace([np.inf, -np.inf], 0).fillna(0)


In [ ]:
X_trainlr2, X_testlr2, y_trainlr2, y_testlr2 = train_test_split(Xlr2, ylr2, test_size=0.2, random_state=42)

In [ ]:
#Checking the predictability of the model with this alpha = 1
EN = ElasticNet(1, l1_ratio=.5)
LR = EN.fit(X_trainlr2, y_trainlr2)
rsq = LR.score(X_testlr2, y_testlr2)
adj_rsq = 1 - (1-rsq)*(len(ylr2)-1)/(len(ylr2)-Xlr2.shape[1]-1)

preds = LR.predict(X_testlr2)
rmse = np.sqrt(MSE(preds, y_testlr2))

print(rsq)
print(adj_rsq)
print(rmse)

### Modeling (Splitting by state-size)

*Didn't end up helping much. Let's just go back to the one with the full train set*

**Grouped states by electoral college vote:**<br>

**Smallest:** <br>
- *3 Electoral votes:* Alaska, Delaware, Montana, North Dakota, South Dakota, Vermont, Wyoming
- *4 Electoral votes:* Hawaii, Idaho, Maine, New Hampshire, Rhode Island
- *5 Electoral votes:* Nebraska, New Mexico, West Virginia

**Medium:** <br>
- *6 Electoral votes:* Arkansas, Iowa ,Kansas, Mississippi, Nevada, Utah
- *7 Electoral votes:* Connecticut, Oklahoma, Oregon
- *8 Electoral votes:* Kentucky, Louisiana
- *9 Electoral votes:* Alabama, Colorado, South Carolina
- *10 Electoral votes:* Maryland, Minnesota, Missouri, Wisconsin
- *11 Electoral votes:* Arizona, Indiana, Massachusetts, Tennessee

**Medium-Large:** <br>
- *12 Electoral votes:* Washington 
- *13 Electoral votes:* Virginia
- *14 Electoral votes:* New Jersey
- *15 Electoral votes:* North Carolina
- *16 Electoral votes:* Georgia, Michigan
- *18 Electoral votes:* Ohio

**Large:** <br>
- *20 Electoral votes:* Illinois, Pennsylvania
- *29 Electoral votes:* Florida, New York
- *38 Electoral votes:* Texas
- *55 Electoral votes:* California


#### Small States

In [ ]:
low_density = ['Alaska', 'Delaware', 'Montana', 'North Dakota', 'South Dakota', 'Vermont', 'Wyoming', 'Hawaii', 'Idaho', 'Maine', 'New Hampshire', 'Rhode Island', 'Nebraska', 'New Mexico', 'West Virginia']

small = hm_model_ready2[hm_model_ready2['STATE'].isin(low_density)].drop('Third Party Votes',axis = 1)

In [ ]:
# small = small.drop(small[small['Primary Loser Vote'] < 1].index)

In [ ]:
small.describe()

**Splitting X&y**

In [ ]:
ysmall = small['GENERAL VOTES'] 
Xsmall = small.drop('GENERAL VOTES',axis = 1)

**Splitting for T-T-S**

In [ ]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(Xsmall, ysmall, test_size=0.3)

**Running Catboost**

In [ ]:
from catboost import CatBoostRegressor, Pool

categorical_features_indices = np.where(X_train_s.dtypes != np.float)[0]
train_pool = Pool(X_train_s, y_train_s, cat_features=categorical_features_indices)
test_pool = Pool(X_test_s, y_test_s, cat_features=categorical_features_indices)


Cat_s = Catset.fit(X_train_s, y_train_s, cat_features=categorical_features_indices , use_best_model=True);

In [ ]:
#Would like to get it down to less than 20,000...
Cat_s.score(X_test_s, y_test_s)

In [ ]:
## Checking out feature importance for this model. 

feature_importances = Cat_s.get_feature_importance(train_pool)
feature_names = X_train_s.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
Cat_s.eval_metrics(data = test_pool, metrics =['R2'], plot = True)

#### Medium States

In [ ]:
med_density = ['Arkansas', 'Iowa', 'Kansas', 'Mississippi', 'Nevada', 'Utah', 'Connecticut', 'Oklahoma', 'Oregon','Kentucky', 'Louisiana','Alabama', 'Colorado', 'South Carolina', 'Maryland', 'Minnesota','Missouri', 'Wisconsin','Arizona','Indiana', 'Massachusetts', 'Tennessee']

med = hm_model_ready2[hm_model_ready2['STATE'].isin(med_density)].drop('Third Party Votes',axis = 1)

In [ ]:
med.describe()

In [ ]:
ymed = med['GENERAL VOTES'] 
Xmed = med.drop('GENERAL VOTES',axis = 1)

In [ ]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(Xmed, ymed, test_size=0.3)

In [ ]:
from catboost import CatBoostRegressor, Pool

categorical_features_indices = np.where(X_train_m.dtypes != np.float)[0]
train_pool = Pool(X_train_m, y_train_m, cat_features=categorical_features_indices)
test_pool = Pool(X_test_m, y_test_m, cat_features=categorical_features_indices)

Cat_m = Catset.fit(X_train_m, y_train_m, cat_features=categorical_features_indices , use_best_model=True);

In [ ]:
#Would like to get it down to less than 20,000...
Cat_m.score(X_test_m, y_test_m)

#### Medium-Large State

In [ ]:
medlar_density = ['Washington','Virginia','New Jersey','North Carolina','Georgia','Michigan','Ohio']

medlar = hm_model_ready2[hm_model_ready2['STATE'].isin(medlar_density)].drop('Third Party Votes',axis = 1)

In [ ]:
medlar.describe()

In [ ]:
ymedlar = medlar['GENERAL VOTES'] 
Xmedlar = medlar.drop('GENERAL VOTES',axis = 1)

In [ ]:
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(Xmedlar, ymedlar, test_size=0.3)

In [ ]:
from catboost import CatBoostRegressor, Pool

categorical_features_indices = np.where(X_train_m.dtypes != np.float)[0]
train_pool = Pool(X_train_ml, y_train_ml, cat_features=categorical_features_indices)
test_pool = Pool(X_test_m, y_test_m, cat_features=categorical_features_indices)

Cat_ml = Catset.fit(X_train_ml, y_train_ml, cat_features=categorical_features_indices , use_best_model=True);

In [ ]:
Cat_ml.score(X_test_ml, y_test_ml)

In [ ]:
Cat_ml.eval_metrics(data = test_pool, metrics =['R2'], plot = True)

### Breaking down by average voter turnout of the past 10 years

In [ ]:
voter_turnout = hm_model_ready2.groupby(['STATE']).mean().reset_index()
voter_turnout = voter_turnout.sort_values(by=['GENERAL VOTES'])

In [ ]:
voter_turnout

In [ ]:
plt.barh(voter_turnout['STATE'],voter_turnout['GENERAL VOTES'], color = '#1A62A5')
plt.ylabel('State', fontsize=14)
plt.xlabel('General Election Votes', fontsize=14)
plt.title('Voter Turnout Per State', fontsize=20)

**Grouped states by voter turnout from the past ten years:**<br>

**Smallest:** <br>
- Wyoming, Alaska, Vermont, Delaware, South Dakota, North Dakota, Hawaii, Rhode Island, 

**Medium:** <br>
- *6 Electoral votes:* Arkansas, Iowa ,Kansas, Mississippi, Nevada, Utah
- *7 Electoral votes:* Connecticut, Oklahoma, Oregon
- *8 Electoral votes:* Kentucky, Louisiana
- *9 Electoral votes:* Alabama, Colorado, South Carolina
- *10 Electoral votes:* Maryland, Minnesota, Missouri, Wisconsin


In [ ]:
low_turnout = ['Wyoming', 'Alaska', 'Vermont', 'Delaware', 'South Dakota', 'North Dakota', 'Hawaii', 'Rhode Island']

low = hm_model_ready2[hm_model_ready2['STATE'].isin(low_turnout)].drop('Third Party Votes',axis = 1)

In [ ]:
low.describe()

**Splitting X&y**

In [ ]:
ylow = low['GENERAL VOTES'] 
Xlow = low.drop('GENERAL VOTES',axis = 1)

**T-T-S**

In [ ]:
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(Xlow, ylow, test_size=0.3)

In [ ]:
from catboost import CatBoostRegressor, Pool

categorical_features_indices = np.where(X_train_l.dtypes != np.float)[0]
train_pool = Pool(X_train_l, y_train_l, cat_features=categorical_features_indices)

Cat_l = Catset.fit(X_train_l, y_train_l, cat_features=categorical_features_indices , use_best_model=True);

In [ ]:
#Would like to get it down to less than 20,000...
Cat_l.score(X_test_l, y_test_l)

### Pipeline for Prediction

In [ ]:
# submission = pd.DataFrame()
# submission['Item_Identifier'] = test['Item_Identifier']
# submission['Outlet_Identifier'] = test['Outlet_Identifier']
# submission['Item_Outlet_Sales'] = model.predict(test)
# submission.to_csv("Submission.csv")

In [ ]:
# Used in prediction

hm_model_ready2

In [ ]:
Cat.predict([['10000', '3', '1', 7000, 'Alaska','2014', 'R',3000,1]])

In [ ]:
#Read in CSV
Primaries_2018 = pd.read_csv('Primary_Results_Predicting.csv')

In [ ]:
#Assigning a year
Primaries_2018['YEAR'] = '2018'

In [ ]:
#IDs to merge on later
Primaries_2018["ID"] = Primaries_2018["State"].map(str) + Primaries_2018["YEAR"] +Primaries_2018["Party"]

In [ ]:
Primaries_party = Primaries_2018[Primaries_2018['Party'].isin(['R','D'])]

** Totaling the Number of votes in primary **

In [ ]:
primaries_partytotal = Primaries_party[['ID','Number of Votes']]

primaries_partytotal = primaries_partytotal.dropna()

primaries_partytotal["Number of Votes"]= primaries_partytotal["Number of Votes"].astype(float)

primaries_partytotal = primaries_partytotal.groupby(['ID']).sum().reset_index()

primaries_partytotal = primaries_partytotal.rename(columns = {'Number of Votes':'Total Party Votes'})

** Counting the Number of Candidates **

In [ ]:
NumRunning_p = Primaries_party.groupby(['ID']).count().reset_index()

NumRunning_p = NumRunning_p[['ID','Number of Votes','(I)']]

NumRunning_p['(I)'].value_counts()

NumRunning_p = NumRunning_p.rename(columns = {'Number of Votes':'Candidate Count'})

** Max votes per party per state **

In [ ]:
PartyWin_p = Primaries_party[['ID','Number of Votes','State','YEAR','Party']]

PartyWin_p = PartyWin_p.dropna()

PartyWin_p["Number of Votes"]= PartyWin_p["Number of Votes"].astype(float)

PartyWin_p = PartyWin_p.groupby(['ID']).max().reset_index()

PartyWin_p = PartyWin_p.rename(columns = {'Number of Votes':'Primary Winner'})

** Merging columns back together **

In [ ]:
votes_merged_p = primaries_partytotal.merge(NumRunning_p, left_on='ID', right_on='ID', how='left')

votes_merged_p = votes_merged_p.merge(PartyWin_p, left_on='ID', right_on='ID', how = 'left')

** Counting total votes by loser **

In [ ]:
votes_merged_p['Primary Loser Vote'] = votes_merged_p['Total Party Votes'] - votes_merged['Primary Winner']

In [ ]:
votes_merged_p

** Adding if party is in the white house **

In [ ]:
OP2018 = []

for party in votes_merged_p['Party']:
    if party == 'R':
        OP2018.append(1)
        
    else:
        OP2018.append(0)
    
votes_merged_p['OFFICE PARTY'] = OP2018

** Seperating out Unopposed **

In [ ]:
Unopp2018 = votes_merged_p.drop(votes_merged_p[votes_merged_p['Primary Winner'] > 1].index)

In [ ]:
Unopp2018

** The races we can do estimates for **

In [ ]:
fin2018 = votes_merged_p.drop(votes_merged_p[votes_merged_p['Primary Winner'] < 1].index)

In [ ]:
fin2018

** Running those estimates!!! ** <3

In [ ]:

GENERAL VOTES	Total Party Votes	Candidate Count	(I)	Primary Winner	STATE	YEAR	PARTY	Primary Loser Vote	Party In House

In [ ]:
fin2018 = fin2018.rename(index=str, columns={"State": "STATE", "Party": "PARTY","OFFICE PARTY":'Party In House'}).drop("ID",axis=1)

# fin2018['Total Party Votes'] = fin2018['Total Party Votes']
# fin2018['Candidate Count'] = fin2018['Candidate Count']
# fin2018['(I)'] = fin2018['(I)']
# fin2018['Primary Winner'] = fin2018['Primary Winner']
# fin2018['STATE'] = fin2018['State']
# fin2018['YEAR'] = fin2018['YEAR']
# fin2018['PARTY'] = fin2018['Party']
# fin2018['Primary Loser Vote'] = fin2018['Primary Loser Vote']
# fin2018['Party In House'] = fin2018['OFFICE PARTY']

# submission.to_csv("Submission.csv")


# fin2018['Predicted Turnout'] = Cat.predict(fin2018)
# submission.to_csv("Submission.csv")
# Cat.predict([['10000', '3', '1', 7000, 'Alaska','2014', 'R',3000,1]])

In [ ]:
fin2018['Predicted Turnout'] = Cat.predict(fin2018)

In [ ]:
fin2018

**Merging back with ID**